# 第1章 好的推荐系统
## 推荐系统测评
### 实验方法
- 离线实验
- 用户调查
- 在线实验

### 测评指标
- 用户满意度
- 预测准确度
    + 评分预测
    + TopN推荐
- 覆盖率
- 多样性
- 相似性
- 新颖性
- 惊喜度
- 信任度
- 实时性
- 健壮性

### 维度测评
- 用户维度
- 物品维度
- 时间维度


# 第2章 利用用户行为数据

- 无上下文信息的隐性反馈数据集：每一条记录仅仅包含用户ID和物品ID

- 无上下文信息的显性反馈数据集：每一条记录包含用户ID、物品ID和用户对物品的评分

- 有上下文信息的隐性反馈数据集：每一条记录包含用户ID、物品ID和用户对物品的评分和评分行为发生的时间戳

仅仅基于用户行为数据设计的推荐算法成为协同过滤算法，具体细分为基于邻域的方法、隐语义模型、基于图的随机游走算法。目前业界应用较为广泛的方法是基于邻域的方法，基于邻域的方法包括下面两种算法：

- 基于用户的协同过滤算法

- 基于物品的协同过滤算法

## 基于用户的协同过滤算法

1. 找到和目标用户兴趣相似的用户集合

2. 将兴趣相似用户喜欢且目标用户不曾见过的物品推荐给目标用户

In [1]:
import math
def UserSimilarity(train):
    res = {}
    for u in train:
        res.setdefault(u,0)
        for v in train:
            if u != v:
                res[u].setdefault(v,[])
                res[u][v] = len(train[u] & train[v])
                res[u][v] /= math.sqrt( len(train[u]) * len(train[v]))
    return res

In [2]:
from collections import defaultdict
def UserSimilarity(train):
    res = defaultdict(dict)
    for u in train:
        for v in train:
            if u != v:
                # 以余弦相似度为计算核心
                res[u][v] = len(train[u] & train[v])
                res[u][v] /= math.sqrt( len(train[u]) * len(train[v]))
    return res

In [3]:
train_data = {'A':['a','b','c'],'B':['a','c']} # 会报错
# train_data = {'A':{'a','b','c'},'B':{'a','c'}} # 只能用set结构，用set结构也比较合理
train_data = {
    'A':{'a','b','d'},
    'B':{'a','c'},
    'C':{'b','e'},
    'D':{'c','d','e'}
}
UserSimilarity(train_data)

defaultdict(dict,
            {'A': {'B': 0.4082482904638631,
              'C': 0.4082482904638631,
              'D': 0.3333333333333333},
             'B': {'A': 0.4082482904638631, 'C': 0.0, 'D': 0.4082482904638631},
             'C': {'A': 0.4082482904638631, 'B': 0.0, 'D': 0.4082482904638631},
             'D': {'A': 0.3333333333333333,
              'B': 0.4082482904638631,
              'C': 0.4082482904638631}})

### 看升级！！！

In [4]:
def UserSimilarity(train):
    item_user = defaultdict(set)
    # item_user = defaultdict(list)
    # item_user = defaultdict(dict)
    for user, items in train.items():
        for item in items:
            # item_user[item].append(user)
            item_user[item].add(user)
    print('用户-物品转置表：',item_user)
    co_num = defaultdict(dict)
    # user_num = {}
    user_num = defaultdict(int)
    for item, users in item_user.items():
        for u in users:
            user_num.setdefault(user,0)
            user_num[u] += 1
            for v in users:
                co_num[u].setdefault(v,0)
                if u != v:
                    # co_num[u][v] += 1
                    co_num[u][v] += 1 / math.log( 1+len(users))
    print('单个用户评价的物品个数', user_num)        
    print('协同用户评价的物品个数', co_num)
    
    res = defaultdict(dict)
    for user, co_user in co_num.items():
        for u,n in co_user.items():
            res[user][u] = n
            res[user][u] /= math.sqrt(user_num[user] * user_num[u])
    print('最终结果-用户相似度', res)
    return res
user_similarity_res = UserSimilarity(train_data)
user_similarity_res

用户-物品转置表： defaultdict(<class 'set'>, {'d': {'A', 'D'}, 'a': {'B', 'A'}, 'b': {'C', 'A'}, 'c': {'B', 'D'}, 'e': {'C', 'D'}})
单个用户评价的物品个数 defaultdict(<class 'int'>, {'D': 3, 'A': 3, 'B': 2, 'C': 2})
协同用户评价的物品个数 defaultdict(<class 'dict'>, {'A': {'A': 0, 'D': 0.9102392266268373, 'B': 0.9102392266268373, 'C': 0.9102392266268373}, 'D': {'A': 0.9102392266268373, 'D': 0, 'B': 0.9102392266268373, 'C': 0.9102392266268373}, 'B': {'B': 0, 'A': 0.9102392266268373, 'D': 0.9102392266268373}, 'C': {'C': 0, 'A': 0.9102392266268373, 'D': 0.9102392266268373}})
最终结果-用户相似度 defaultdict(<class 'dict'>, {'A': {'A': 0.0, 'D': 0.3034130755422791, 'B': 0.37160360818355515, 'C': 0.37160360818355515}, 'D': {'A': 0.3034130755422791, 'D': 0.0, 'B': 0.37160360818355515, 'C': 0.37160360818355515}, 'B': {'B': 0.0, 'A': 0.37160360818355515, 'D': 0.37160360818355515}, 'C': {'C': 0.0, 'A': 0.37160360818355515, 'D': 0.37160360818355515}})


defaultdict(dict,
            {'A': {'A': 0.0,
              'D': 0.3034130755422791,
              'B': 0.37160360818355515,
              'C': 0.37160360818355515},
             'D': {'A': 0.3034130755422791,
              'D': 0.0,
              'B': 0.37160360818355515,
              'C': 0.37160360818355515},
             'B': {'B': 0.0,
              'A': 0.37160360818355515,
              'D': 0.37160360818355515},
             'C': {'C': 0.0,
              'A': 0.37160360818355515,
              'D': 0.37160360818355515}})

In [5]:
def recommend(user, train, user_similarity):
    rank = dict()
    user_item = train[user]
    for u,s in sorted(user_similarity[user].items(), key=lambda x:x[1], reverse=True):
        for item in train[u]:
            rank.setdefault(item, 0)
            if item not in user_item:
                rank[item] += s
    return rank
recommend('A', train_data, user_similarity_res)

{'a': 0, 'c': 0.6750166837258342, 'e': 0.6750166837258342, 'b': 0, 'd': 0}

# 看缺点

基于用户的协同过滤算法有如下缺点：

1. 随着网站用户数目越来越大，计算用户的相似度越来越困难

2. 基于用户的协同过滤算法很难对推荐结果做出解释

因此，基于物品的协同过滤算法应运而生了。简称ItemCF，但是ItemCF并不利用物品的内容属性计算物品之间的相似度，它主要通过分析用户的行为记录计算物品之间的相似性。

In [6]:
def ItemSimilarity(train):
    item_user = defaultdict(set)
    for user,items in train.items():
        for item in items:
            item_user[item].add(user)
    return item_user
UserSimilarity(ItemSimilarity(train_data))


用户-物品转置表： defaultdict(<class 'set'>, {'A': {'d', 'a', 'b'}, 'D': {'d', 'e', 'c'}, 'B': {'a', 'c'}, 'C': {'e', 'b'}})
单个用户评价的物品个数 defaultdict(<class 'int'>, {'e': 2, 'd': 2, 'a': 2, 'b': 2, 'c': 2})
协同用户评价的物品个数 defaultdict(<class 'dict'>, {'d': {'d': 0, 'a': 0.7213475204444817, 'b': 0.7213475204444817, 'e': 0.7213475204444817, 'c': 0.7213475204444817}, 'a': {'d': 0.7213475204444817, 'a': 0, 'b': 0.7213475204444817, 'c': 0.9102392266268373}, 'b': {'d': 0.7213475204444817, 'a': 0.7213475204444817, 'b': 0, 'e': 0.9102392266268373}, 'e': {'d': 0.7213475204444817, 'e': 0, 'c': 0.7213475204444817, 'b': 0.9102392266268373}, 'c': {'d': 0.7213475204444817, 'e': 0.7213475204444817, 'c': 0, 'a': 0.9102392266268373}})
最终结果-用户相似度 defaultdict(<class 'dict'>, {'d': {'d': 0.0, 'a': 0.36067376022224085, 'b': 0.36067376022224085, 'e': 0.36067376022224085, 'c': 0.36067376022224085}, 'a': {'d': 0.36067376022224085, 'a': 0.0, 'b': 0.36067376022224085, 'c': 0.45511961331341866}, 'b': {'d': 0.3606737602222408

defaultdict(dict,
            {'d': {'d': 0.0,
              'a': 0.36067376022224085,
              'b': 0.36067376022224085,
              'e': 0.36067376022224085,
              'c': 0.36067376022224085},
             'a': {'d': 0.36067376022224085,
              'a': 0.0,
              'b': 0.36067376022224085,
              'c': 0.45511961331341866},
             'b': {'d': 0.36067376022224085,
              'a': 0.36067376022224085,
              'b': 0.0,
              'e': 0.45511961331341866},
             'e': {'d': 0.36067376022224085,
              'e': 0.0,
              'c': 0.36067376022224085,
              'b': 0.45511961331341866},
             'c': {'d': 0.36067376022224085,
              'e': 0.36067376022224085,
              'c': 0.0,
              'a': 0.45511961331341866}})

In [7]:
def ItemSimilarity(train):
    goods = {}
    co_goods = defaultdict(dict)
    for user,items in train.items():
        for item1 in items:
            goods.setdefault(item1,0)
            goods[item1] += 1
            for item2 in items:
                co_goods[item1].setdefault(item2,0)
                if item1 != item2:
                    co_goods[item1][item2] += 1
    res = defaultdict(dict)
    
    for item,co_item in co_goods.items():
        for k,v in co_item.items():
            res[item][k] = v/math.sqrt( goods[item] * goods[k])
    return res

train_data = {
    'A':{'a','b','d'},
    'B':{'a','c'},
    'C':{'b','e'},
    'D':{'c','d','e'},
    'E':{'a','d'}
}

item_similarity = ItemSimilarity(train_data)
item_similarity

defaultdict(dict,
            {'d': {'d': 0.0,
              'a': 0.6666666666666666,
              'b': 0.4082482904638631,
              'e': 0.4082482904638631,
              'c': 0.4082482904638631},
             'a': {'d': 0.6666666666666666,
              'a': 0.0,
              'b': 0.4082482904638631,
              'c': 0.4082482904638631},
             'b': {'d': 0.4082482904638631,
              'a': 0.4082482904638631,
              'b': 0.0,
              'e': 0.5},
             'c': {'a': 0.4082482904638631,
              'c': 0.0,
              'd': 0.4082482904638631,
              'e': 0.5},
             'e': {'e': 0.0, 'b': 0.5, 'd': 0.4082482904638631, 'c': 0.5}})

In [10]:
def recommendation(train, user, item_similarity):
    rank = defaultdict(dict)
    user_item = train[user]
    for item in user_item:
        for co_item,score in sorted(item_similarity[item].items(),key=lambda x:x[1], reverse=True):
            rank.setdefault(co_item,0)
            if co_item not in user_item:
                rank[co_item] += score
    return rank
recommendation(train_data, 'A', item_similarity)

defaultdict(dict,
            {'a': 0,
             'b': 0,
             'e': 0.9082482904638631,
             'c': 0.8164965809277261,
             'd': 0})

## 隐语义模型 Latent factor model

推荐系统的用户行为分为显性反馈和隐性反馈。
正样本：用户喜欢什么物品
负样本：用户不喜欢什么物品

对负样本采样应该遵循以下原则：
1. 对每个样本，要保证正负样本的平衡（数目相似）
2. 对每个用户采样负样本时，要选取那些很热门，而用户却没有行为的物品

In [ ]:
def RandomSelectNegativeSample(self, items):
    

In [ ]:
def LatentFactorModel(user_items, F, N, alpha, lambda):


# 第3章 推荐系统冷启动问题

冷启动问题（cold start）主要分3类：
1. 用户冷启动：解决如何给新用户做个性化推荐
2. 物品冷启动：解决如何将新的物品推荐给可能对它感兴趣的用户
3. 系统冷启动：解决上述两个问题

So 如何解决呢？
1. 提供非个性化推荐：例如热门排行榜
2. 利用注册信息提供粗粒度的推荐
3. 利用用户的社交网络，导入社交好友信息，推荐其好友喜欢的物品
4. 用户登录时强制对一些物品反馈
5. 对于新加入的物品，根据物品相似度进行推荐
6. 引入专家知识，构建物品相关度表

# 第4章 利用用户标签数据

基于标签的推荐系统算法：
1. 统计每个用户最常用的标签
2. 对于每个标签，统计被打过这个标签次数最多的物品
3. 对于每个用户，首先在【1】中找到他最常用的标签，推荐【2】中的物品给这个用户



In [25]:
'''
users_tags = {
    'userA':{'sports':2,'art':1,'science':3},
    'userB':{'sports':1,'art':3}
}

tags_items = {
    'sports':{'Kobe':2, 'Yaoming':1,'Taylor':0},
    'art':{'Taylor':2},
    'science':{'Yang_zhen_ning':2}
}
'''
from collections import defaultdict
import math
def recommend(user, users_tags, tags_items):
    # 统计 标签 被 用户使用的情况
    tag_num = defaultdict(dict)
    for user,tags in users_tags.items():
        for tag,_ in tags.items():
            tag_num.setdefault(tag, 0)
            tag_num[tag] += 1
    
    recommend_item = defaultdict(dict)
    user_tags = users_tags[user]
    for tag_item,tag_count in users_tags[user].items():
        #3 此处添加有关标签相似性的调用程序，返回与用户当前标记的标签相似度高的标签进行拓展推荐
        for item,item_count in tags_items[tag_item].items():
            recommend_item.setdefault(item, 0)
            #1 标签数量 * 物品数量
            #recommend_item[item] += tag_count * item_count
            #2
            # 惩罚频繁使用的标签
            recommend_item[item] += tag_count * item_count / math.log(1 + tag_num[tag_item])
    return recommend_item

users_tags = {
    # 用户：{标签：数量}
    'userA':{'sports':2,'art':1,'science':3},
    'userB':{'sports':1,'art':3}
}

tags_items = {
    # 标签：{物品：数量}
    'sports':{'Kobe':2, 'Yaoming':1},
    'art':{'Taylor':2},
    'science':{'Yang_zhen_ning':2}
}

recommend('userB', users_tags, tags_items)

defaultdict(dict,
            {'Kobe': 1.8204784532536746,
             'Yaoming': 0.9102392266268373,
             'Taylor': 5.461435359761024})

## 看改进

根据【标签出现频率】和【物品被贴标签的频率】（即：#1方法）来作为推荐的权重反馈给用户的最大问题是：容易将热门产品推荐给用户，降低推荐结果的新颖性。需要进行改进：

1. TF-IDF：对标签被用户使用的频率进行惩罚（即：#2）、对热门物品进行惩罚
2. 数据稀疏性：用户的兴趣与推荐的物品是通过标签进行联系的，但是对于新用户或新物品，标签数量会很少！解决方法就是拓展标签。
如何拓展呢？有二：
    - 根据标签的同义词字典
    - 根据标签的相似性（余弦相似度等）见上文的#3
        - 计算原理：当两个标签同时出现在很多物品的标签集合中，就可以认为两个标签具有较大相似性
        - 基于余弦相似度的标签相似度计算公式：
            1. 遍历每一个标签，统计每个标签下的物品集合，记为N(b)
            2. $n_{b,i}$表示给物品i打上标签b的用户数
            3. 以物品i为纽带,计算两个标签之间的相似度：$sim(b,b') = \frac{\sum_{i \in N(b) \bigcap N(b')} n_{b,i} n_{b',i}}{\sqrt{ \sum_{i \in N(b)} n_{b,i}^2 \sum_{i \in N(b') n_{b',i}^2} } } $
3. 标签清理：
    - 去除词频很高的停用词
    - 去除因词根不同造成的同义词
    - 去除因分隔符不同造成的同义词

# 第5章 利用上下文信息

## 时间效应
- 用户兴趣是变化的
- 物品是有生命周期的
- 季节效应

Q：如何保证每天给用户推荐的结果都不一样（很大程度上不一样）？
A：将该问题转化为保证给用户的推荐结果具有一定的时间多样性：
 - 在生成推荐结果时加入一定的随机性。比如从推荐列表前20个结果中随机挑选10个结果展示给用户
 - 记录用户每天看到的推荐结果，然后在每天给用户进行推荐时，对他前几天看到过很多次的推荐结果进行降权
 - 每天给用户使用不同的推荐算法，比如随机使用协同过滤算法、内容过滤算法
 
## 时间上下文推荐算法

包含时间信息的用户行为数据集由一系列三元组构成，三元组(u,t,i)表示用户u在时刻t对物品i产生过行为

1. 最近最热门

$$ n_i(T) = \sum_{(u,t,i) \in Train, t < T } \frac{1}{1 + \alpha(T - t)} $$

需要注意时间的转换：

```python
def recent_popular(records, alpha, T):
    res = {}
    for user,item,time in records:
        for time < T:
            res[item] = 1 / (1.0 + alpha*(T - time))
    return res
```

2. 时间上下文相关的ItemCF算法

算法核心：
- 利用用户行为离线计算物品之间的相似度
- 根据用户的历史行为和物品相似度矩阵，对用户进行个性化推荐

时间信息在上面两个核心部分都有体现：

- 物品相似度：用户在相隔很短的时间内喜欢的物品具有很高相似度。如用户今天看的电影和用户昨天看的电影的相似度在统计意义上要远大于今天看的电影和去年看的电影的相似度
- 在线推荐：用户近期行为相比用户很久之前的行为，更能体现用户现在的兴趣

In [41]:
from collections import defaultdict
import math

def item_similarity(train, alpha=0.2):
    item_count = {}
    item_similarity_matrx = defaultdict(dict)
    for user, item, time in train:
        item_count.setdefault(item, 0)
        item_count[item] += 1
        for user_1, item_1, time_1 in train:
            item_similarity_matrx[item].setdefault(item_1, 0)
            if item != item_1:
                # 时间衰减函数
                # 时间距离相隔越远，则权重越小
                # alpha 是时间衰减参数
                # 如果一个系统用户兴趣变化较快，则alpha的值应该较大
                item_similarity_matrx[item][item_1] += 1 / (1 + alpha * abs(int(time_1) - int(time)))
    # print(item_count)
    # print(item_similarity_matrx)
    res = defaultdict(dict)
    for item,items in item_similarity_matrx.items():
        for sim_item,val in items.items():
            res[item][sim_item] = val / math.sqrt( item_count[item] * item_count[sim_item])
    return res


def recommendation(train, user_id, item_similarity, K, t0, alpha=0.3):
    '''
    t0:当前时间
    '''
    user_id_item = defaultdict(dict)
    for user,item,time in train:
        user_id_item[user][item] = time
    
    target_user_item = user_id_item[user_id]
    rank = defaultdict(dict)
    for item,time in user_id_item[user_id].items():
        for item_sim,weight in sorted(item_similarity[item].items(),key =lambda x:x[1], reverse=True)[:K]:
            rank.setdefault(item_sim,0)
            if item_sim not in target_user_item:
                rank[item_sim] += weight / (1+alpha* (t0 - float(time)))
    return sorted(rank.items(),key=lambda x:x[1],reverse=True)
    
train_data_list = [
    ('A','a','8'),
    ('A','b','7'),
    ('A','d','6'),
    ('B','a','9'),
    ('B','c','8'),
    ('C','c','8'),
    ('C','a','8'),
    ('C','e','8'),
    ('D','a','7'),
    ('D','c','9'),
]

item_similarity = item_similarity(train_data_list,alpha=0.8)
recommendation(train_data_list,'B',item_similarity,4,10)

[('e', 2.1187378765368488),
 ('b', 1.4996186153872446),
 ('d', 1.0063581827785695),
 ('c', 0),
 ('a', 0)]

## 时间上下文相关的UserCF算法

算法核心：
- 计算用户相似度，考虑时间因素在内
- 根据用户相似度，推荐相似用户喜欢且目标用户不曾看过的

In [82]:
train_data_list = [
    ('A','a','8'),
    ('A','b','7'),
    ('A','d','6'),
    ('B','a','9'),
    ('B','c','8'),
    ('D','a','10'),
    ('D','c','9'),
    ('C','c','8'),
    ('C','a','8'),
    ('C','e','8'),
]

def user_similarity(train,alpha=0.3):
    item_users = defaultdict(dict)
    user_items = defaultdict(dict)
    for user,item,time in train:
        item_users[item][user] = time
        user_items[user][item] = time
    
    user_count = defaultdict(dict)
    user_matrx = defaultdict(dict)
    for item,users in item_users.items():
        for user,time in users.items():
            user_count.setdefault(user,0)
            user_count[user] += 1
            for user_1,time_1 in users.items():
                user_matrx[user].setdefault(user_1,0)
                if user != user_1:
                    # user_matrx[user][user_1] += 1
                    user_matrx[user][user_1] += 1 / (1 + alpha * abs(float(time) - float(time_1)))
    # print(user_matrx)
    res = defaultdict(dict)
    for user,sim_users in user_matrx.items():
        for sim_user,weight in sim_users.items():
            res[user][sim_user] = weight / math.sqrt(user_count[user] * user_count[sim_user])
    return item_users,user_items,res

item_users,user_items,user_similarity = user_similarity(train_data_list)

# print(item_users,user_items,user_similarity)
# print(user_items)

def recommendation(train,user_id, user_similarity,k,alpha=0.3,t0=10):
    rank = defaultdict(dict)
    target_user_items = user_items[user_id]
    for user,weight in sorted(user_similarity[user_id].items(), key=lambda x:x[1],reverse=True)[:k]:
        for item,time in user_items[user].items():
            if item not in target_user_items:
                rank[item] = weight / (1 + alpha * (t0 - float(time)))
    return rank
recommendation(train_data,'A', user_similarity,3,alpha=0.3)

defaultdict(dict, {'c': 0.1962732165691649, 'e': 0.20833333333333331})

# 第6章 利用社交网络数据

# 第7章 推荐系统实例

## 7.3 生成用户特征向量

一般来说，用户特征向量包括两种：
- 用户的人口统计学特征
- 用户的行为特征

用户的行为特征需要考量以下因素：

- 用户行为种类
 - 浏览商品，点击商品，收藏商品，购买商品等，这些行为的权重应该是不一样的
- 用户行为产生时间
 - 近期行为的权重要高于过往的行为
- 用户行为的次数
 - 用户反复对同一个物品进行操作，则该物品的权重也应该较高
- 物品的热门程度
 - 如果用户对一个热门的物品产生行为，那不能代表用户的个性；如果用户对非热门的物品产生行为，则该物品的权重要适当提高
 
### 过滤模块
- 用户已经产生过行为的物品
- 候选物品以外的物品
- 某些质量很差的物品

# 第8章 评分预测问题